In [1]:
import requests
import json


In [2]:
def getToken(username, password):
    res = requests.post(
        "http://localhost:8000/get-token/",
        data={"username": "bbb", "password": "."},
        verify=False,
    )
    return res.json()["token"]


def makeRoom(token, title, subtitle=None):
    res = requests.post(
        "http://localhost:8000/rooms/",
        headers={"Authorization": "Token " + token},
        data={"subtitle": subtitle, "title": title},
    )
    return res


def makeEvent(token, room, event_type, args, parent_event=None):
    data = {"room": room, "event_type":event_type, "args":args}
    if parent_event:
        data["parent_event"] = parent_event
    res = requests.post(
        "http://localhost:8000/events/",
        headers={"Authorization": "Token " + token, "content-type": "application/json"},
        data=json.dumps(data),
    )
    return res

def makeTune(token):
    with open("test.mp3", 'rb') as fobj:
        res = requests.post(
            "http://localhost:8000/tunes/",
            headers={"Authorization": "Token " + token},
            files={'file': fobj},
        )
        return res
    
def getEvents(token, room, event_type, skip, limit):
    url =  "http://localhost:8000/events/"
    
    res = requests.get(url,
                       params={"room":room, "event_type":event_type, "skip":skip, "limit":limit }, 
                       headers={"Authorization": "Token " + token})
    return res

def getTuneMeta(token, tune_id):
    url = "http://localhost:8000/tunes/{}/meta/".format(tune_id)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res

def getTuneData(token, tune_id):
    url = "http://localhost:8000/tunes/{}/data/".format(tune_id)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res

def getUser(token, skip, limit):
    url = "http://localhost:8000/users/?skip={}&limit={}".format(skip, limit)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res


def makeUser(username, password):
    url = "http://localhost:8000/users/"
    res = requests.post(url,
        data={"username": username, "password": password},
    )
    return res

def getTuneSync(token, room_id):
    url = "http://localhost:8000/rooms/{}/tunesync/".format(room_id)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res

def kickUser(token, room_id, kicked_user):
    return makeEvent(token, room_id, "U", {"type": "K", "user": kicked_user})

def inviteUsers(token, room_id, users):
    return makeEvent(token, room_id, "U", {"type": "I", "users": users})

def changeUser(token, room_id, user, role):
    return makeEvent(token, room_id, "U", {"type": "C", "user": user, "role": role})

def userJoin(token, room_id, is_accepted):
    return makeEvent(token, room_id, "U", {"type": "J", "is_accepted": is_accepted})

def getTune(token, query):
    url = "http://localhost:8000/tunes?{}".format(query)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res


def getRoomEvent(token, room_id):
    url = "http://localhost:8000/rooms/{}/events/".format(room_id)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res

def getRoomTuneSync(token, room_id):
    url = "http://localhost:8000/rooms/{}/tunesync/".format(room_id)
    res = requests.get(url, headers={"Authorization": "Token " + token})
    return res


In [65]:
res = getTune(TOKEN, "album__icontains=royalty")
res.json()

[{'id': 1,
  'name': "['Drums Urban Percussion Intro 2 (Music For Video)']",
  'length': 23.412125}]

In [100]:
res = getRoomTuneSync(TOKEN, 2)
res.json()

{'detail': 'You do not have permission to perform this action.'}

In [5]:
res = getRoomEvent(TOKEN, 3)
res.json()

{'detail': 'You do not have permission to perform this action.'}

In [66]:
res = getTune(TOKEN, "uploader=bbb")
res.json()

[{'id': 1,
  'name': "['Drums Urban Percussion Intro 2 (Music For Video)']",
  'length': 23.412125}]

In [7]:
res = makeUser("krillin", "green123")
res.json()


{'id': 3, 'username': 'krillin'}

In [35]:
makeUser("goku", "green123")

<Response [200]>

In [4]:
USERNAME = "goku"
PASSWORD = "green123"


TOKEN = getToken(USERNAME, PASSWORD)

In [6]:
TOKEN

'c98c9aaa84b750c93fa284c8e24ad867ad34c1b2'

In [8]:
res = makeRoom(TOKEN, "sex")
res.json()

{'id': 2,
 'title': 'sex',
 'subtitle': None,
 'creator': 2,
 'members': [2],
 'system_user': None}

In [9]:
res = makeEvent(TOKEN, 2, "M", {"content": "sex"})
res.json()

{'id': 1,
 'room': 2,
 'creation_time': '2020-03-24T19:19:30.596620Z',
 'event_type': 'M',
 'author': 2,
 'parent_event_id': None,
 'args': {'content': 'sex'},
 'isDeleted': False}

In [ ]:
# django=> select * from tunesync_membership;
#  id | state | role | room_id | user_id 
# ----+-------+------+---------+---------
#   1 | A     | A    |       1 |       3
#   2 | A     | A    |       2 |       2
# (2 rows)


In [68]:
res.json()

{'detail': 'You do not have permission to perform this action.'}

In [24]:
res = makeEvent(TOKEN, 2, "T", {"play":{"timestamp":0.0, 'is_playing': True, 'queue_index': 0}})
res.json()

{'last_modify_queue': {'queue': [[1,
    23.412125,
    "['Drums Urban Percussion Intro 2 (Music For Video)']"]],
  'event_id': 7},
 'last_play': {'timestamp': 0.0,
  'is_playing': True,
  'queue_index': 0,
  'event_id': 9},
 'play_time': '2020-03-24T19:23:03.161059Z',
 'room_id': 2}

In [22]:
res = makeEvent(TOKEN, 2, "T", {"modify_queue":{"queue":[1]}})
res.json()

{'last_modify_queue': {'queue': [[1,
    23.412125,
    "['Drums Urban Percussion Intro 2 (Music For Video)']"]],
  'event_id': 7},
 'last_play': None,
 'play_time': None,
 'room_id': 2}

In [25]:
res = makeEvent(TOKEN, 3, "U", {"type": [1,2]})
res.json()

{'detail': 'You do not have permission to perform this action.'}

In [24]:
res

<Response [200]>

In [47]:
res = makeEvent(TOKEN, 8, "PL", {"song_id": 1, "is_playing":True, "timestamp": 10.4})

In [48]:
res.json()

{'detail': 'You do not have permission to perform this action.'}

In [67]:
res = inviteUsers(TOKEN, 2, [4])
res.json()

{'id': 8,
 'room': 2,
 'creation_time': '2020-03-20T18:57:22.611842Z',
 'event_type': 'U',
 'author': 2,
 'parent_event_id': None,
 'args': {'type': 'I', 'users': [4]},
 'isDeleted': False}

In [39]:
res = inviteUsers(TOKEN, 2, [3])
res.json()

{'id': 19,
 'room': 2,
 'creation_time': '2020-03-24T19:32:49.079127Z',
 'event_type': 'U',
 'author': 2,
 'parent_event_id': None,
 'args': {'type': 'I', 'users': [3]},
 'isDeleted': False}

In [38]:
res = kickUser(TOKEN, 2, 3)
res.json()

{'details': 'user is not in the room'}

In [46]:
res.status_code

400

In [74]:
res = changeUser(TOKEN, 2, 2, "D")
res.json()

{'id': 10,
 'room': 2,
 'creation_time': '2020-03-21T00:08:04.211899Z',
 'event_type': 'U',
 'author': 2,
 'parent_event_id': None,
 'args': {'type': 'C', 'user': 2, 'role': 'D'},
 'isDeleted': False}

In [69]:
res.json()

{'details': 'user is not in the room'}

In [34]:
res = makeEvent(TOKEN, 1, "M", {"content": "sex"})

In [35]:
res.json()

{'id': 9,
 'room': 1,
 'creation_time': '2020-03-16T12:40:21.851273Z',
 'event_type': 'M',
 'author': 2,
 'parent_event_id': None,
 'args': {'content': 'sex'},
 'isDeleted': False}

In [36]:
res = makeEvent(TOKEN, 8, "PO", {"action": "PL", "song_id": 1, "is_playing":True, "timestamp": 10.4})

In [37]:
res.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [48]:
res = makeEvent(TOKEN, 8, "PO", {"action": "MQ", "queue": [1 ,2]})

In [49]:
res.json()

{'id': 39,
 'room': 8,
 'creation_time': '2020-03-15T17:12:06.149680Z',
 'event_type': 'PO',
 'author': 2,
 'parent_event_id': None,
 'args': {'action': 'MQ', 'queue': [1, 2]},
 'isDeleted': False}

In [51]:
res = makeEvent(TOKEN, 8, "PO", {"action": "U", "type": "K" , "user": 4})

In [46]:
res.json()

[{'id': 1,
  'uploader': 2,
  'name': "['Drums Urban Percussion Intro 2 (Music For Video)']",
  'artist': "['TunePocket.com Music Library']",
  'album': "['TunePocket Unlimited Royalty Free Music Library']",
  'mime': 'audio/mp3',
  'length': 23.412125}]

In [19]:
res = makeTune(TOKEN)

In [92]:
res = getTuneMeta(TOKEN, 2)

In [15]:
res = getUser(TOKEN, 0, 5)

In [107]:
res = getTuneData(TOKEN,2)

In [20]:
res.json()

[{'id': 2,
  'uploader': 2,
  'name': "['Drums Urban Percussion Intro 2 (Music For Video)']",
  'artist': "['TunePocket.com Music Library']",
  'album': "['TunePocket Unlimited Royalty Free Music Library']",
  'mime': 'audio/mp3',
  'length': 23.412125}]

In [35]:
audio = mutagen.File("test.mp3", easy=True)
audio.tags

{'album': ['TunePocket Unlimited Royalty Free Music Library'], 'composer': ['www.tunepocket.com'], 'title': ['Drums Urban Percussion Intro 2 (Music For Video)'], 'artist': ['TunePocket.com Music Library'], 'albumartist': ['www.tunepocket.com'], 'genre': ['Electronic']}

In [39]:
audio.mime[0]

'audio/mp3'

In [40]:
audio.info.length

23.412125

In [44]:
audio["title"]

['Drums Urban Percussion Intro 2 (Music For Video)']